In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [2]:
from benchmark.random_k_sat import RandomKSAT

# TODO: Scale up to k = 8, n = 12 and num_instances = 10
k = 3
n = 8
num_instances = 15

instances = RandomKSAT.from_poisson(n, k, instances=num_instances).formulas

In [3]:
from k_sat.quantum_solver.pauli_encoder import PauliEncoder
from k_sat.quantum_solver.average_optimiser import AverageOptimiser

ps = [1] #2, 4, 8, 30, 60]
optimal_params = {}

encoder = PauliEncoder()
optimiser = AverageOptimiser()

for p in ps:

	# Encode instances
	print('encoding circuits')
	training_circuits = [(formula, encoder.encode_formula(formula, p)) for formula in instances]

	# Initialise gamma_i = -0.01, beta_i = 0.01 
	initial_params = [-0.01, 0.01] * p

	# Find and store optimal parameters across instances
	print('finding parameters')
	optimal_params[p] = optimiser.find_optimal_params(initial_params, training_circuits)

encoding circuits
finding parameters


In [4]:
optimal_params

{1: array([0.62742206, 0.38405219])}

In [5]:
instance = RandomKSAT.from_poisson(12, 3, satisfiable=True)

In [6]:
instance.formulas[0]

(x10 ∨ ¬x5 ∨ ¬x6)∧
(x6 ∨ ¬x1 ∨ ¬x3)∧
(¬x7 ∨ ¬x9 ∨ x3)∧
(x8 ∨ x2 ∨ ¬x8)∧
(x8 ∨ ¬x4 ∨ x2)∧
(x11 ∨ x0 ∨ ¬x5)∧
(¬x5 ∨ x10 ∨ x1)∧
(¬x2 ∨ ¬x7 ∨ ¬x9)∧
(x11 ∨ ¬x6)∧
(x10 ∨ ¬x8 ∨ ¬x5)∧
(x0 ∨ x9 ∨ ¬x3)∧
(¬x9 ∨ ¬x2 ∨ ¬x8)∧
(¬x8 ∨ ¬x10)∧
(x0 ∨ x2 ∨ ¬x7)∧
(x4 ∨ x10 ∨ ¬x3)∧
(x1 ∨ x9 ∨ x3)∧
(¬x9 ∨ ¬x6 ∨ ¬x1)∧
(x5 ∨ x2 ∨ x0)∧
(¬x11 ∨ x7 ∨ x9)∧
(x0 ∨ x5 ∨ ¬x0)∧
(x6 ∨ ¬x0 ∨ x8)∧
(¬x8 ∨ x10 ∨ ¬x6)∧
(x4 ∨ x10 ∨ ¬x7)∧
(x3 ∨ x8 ∨ x1)∧
(¬x3 ∨ x2 ∨ ¬x6)∧
(¬x3 ∨ x2 ∨ x0)∧
(x10 ∨ ¬x0 ∨ x0)∧
(x1 ∨ ¬x1 ∨ x9)∧
(x0 ∨ ¬x7 ∨ x3)∧
(x10 ∨ x5)∧
(x3 ∨ ¬x7 ∨ x11)∧
(¬x10 ∨ ¬x0 ∨ x4)∧
(x2 ∨ ¬x7 ∨ x1)∧
(x0 ∨ x1 ∨ ¬x2)∧
(¬x4 ∨ x3 ∨ ¬x0)∧
(¬x11 ∨ ¬x6 ∨ ¬x3)∧
(¬x1 ∨ x4)∧
(¬x9 ∨ ¬x8 ∨ x11)∧
(¬x6 ∨ x2 ∨ x0)∧
(x2 ∨ x5 ∨ ¬x0)∧
(¬x2 ∨ ¬x4 ∨ ¬x9)∧
(x6 ∨ x10 ∨ x9)∧
(¬x10 ∨ ¬x11)∧
(¬x5 ∨ x3 ∨ x0)∧
(x3 ∨ ¬x7 ∨ ¬x3)∧
(x4 ∨ ¬x5 ∨ x7)∧
(x7 ∨ ¬x3 ∨ x2)∧
(x2 ∨ x3 ∨ ¬x10)∧
(x5 ∨ x9 ∨ ¬x6)∧
(¬x10 ∨ ¬x7 ∨ x3)∧
(x11 ∨ ¬x1 ∨ x1)∧
(¬x4 ∨ ¬x5 ∨ ¬x3)∧
(¬x9 ∨ x2 ∨ ¬x0)∧
(x0 ∨ ¬x0 ∨ ¬x11)∧
(x0 ∨ ¬x8)

In [7]:
from k_sat.quantum_solver.qaoa_evaluator import QAOAEvaluator

encoder = PauliEncoder()
evaluator = QAOAEvaluator()

# TODO: evaluate n_e for [12...20], num_instances_e = 10000
k_e = k
n_e = [12]
num_instances_e = 5

# Dictionary results
p_succ = {n : {} for n in n_e}
running_times = {n : {} for n in n_e}

for n in n_e:
	print(f'evaluating n = {n}')
	# Generate satisfiable instances
	instances_e = RandomKSAT.from_poisson(n, k_e, satisfiable=True, instances = num_instances_e)
	for (p, params) in optimal_params.items():
		print(f'evaluating p = {p}')
		p_succ_p = 0
		running_times_p = []
		for formula in instances_e.formulas:
			circuit = encoder.encode_formula(formula, p)
			p_succ_p += evaluator.success_probability(circuit, formula, params)
			running_time_f = evaluator.running_time(circuit, formula, params, timeout=1000) 
			running_times_p.append(running_time_f)


evaluating n = 12
evaluating p = 1


In [8]:
p_succ

{12: {}}

In [9]:
running_times

{12: {}}